In [72]:
## 1. Loading modules 
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
from keras.models import Sequential
from matplotlib import pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import (
train_test_split, StratifiedShuffleSplit)

## 2. Loading and Splitting Data 
# Folder path
path =  "data/"

# Which values are considered NaN
na_vls = ['#N/A', '#N/A N/A', '#NA', '-1.#IND', '-1.#QNAN', '-NaN', '-nan', 
               '1.#IND', '1.#QNAN', '<NA>', 'N/A', 'NA', 'NULL', 'NaN', 'n/a', 'nan', 
                'null', '...']


# Load datasets
df_credit = pd.read_csv(path+"UCI_Credit_Card.csv", na_values = na_vls)

# Handling missing values
df_credit.dropna(inplace=True, how = 'all')
df_credit.fillna(value = '', inplace=True)

# Types for each DataFrame column
crdt_types = {'ID': 'int64',
            'LIMIT_BAL':  'float64',
            'SEX': 'int64',
            'EDUCATION': 'int64',
            'MARRIAGE': 'int64',
            'AGE': 'int64',
            'PAY_0':'int64',
            'PAY_2':'int64',
            'PAY_3':'int64',
            'PAY_4':'int64',
            'PAY_5':'int64',
            'PAY_6':'int64',
            'BILL_AMT1': 'int64',
            'BILL_AMT2': 'int64',
            'BILL_AMT3': 'int64',
            'BILL_AMT4':'int64',
            'BILL_AMT5': 'int64',
            'BILL_AMT6': 'int64',
            'PAY_AMT1': 'int64',
            'PAY_AMT2': 'int64',
            'PAY_AMT3': 'int64',
            'PAY_AMT4':'int64',
            'PAY_AMT5': 'int64',
            'PAY_AMT6': 'int64',
            'default.payment.next.month': 'int64'}

# Casting DataFrames columns according to the types above
df_credit = df_credit.astype(crdt_types)
x = df_credit[['LIMIT_BAL','SEX','EDUCATION','MARRIAGE','AGE',
            'PAY_0','PAY_2','PAY_3','PAY_4','PAY_5','PAY_6',
            'BILL_AMT1','BILL_AMT2','BILL_AMT3',
            'BILL_AMT4','BILL_AMT5','BILL_AMT6',
            'PAY_AMT1','PAY_AMT2','PAY_AMT3','PAY_AMT4',
            'PAY_AMT5','PAY_AMT6']].values
y = df_credit['default.payment.next.month'].values

# Dataset dimensions:
n_data, n_features = x.shape

# Model Selection and Validation Parameters
random_state = 42

# Splitting and Standardization of training and test datasets
std = StandardScaler()

# Train test splitting
test_size = 0.30
train_size = 0.70

# Stratified data splitting into Train Dataset and Test Dataset
sss = StratifiedShuffleSplit(n_splits = 2, test_size = test_size, random_state=random_state)
train_index,test_index = sss.split(x, y)

x_train = x[train_index[0]]
y_train = y[train_index[0]]
x_test = x[test_index[0]]
y_test = y[test_index[0]]


# Data Standardization
x_train_std = std.fit_transform(x_train, y_train)
x_train = x_train_std
x_test_std = std.fit_transform(x_test, y_test)
x_test = x_test_std

In [73]:
batch_size = n_data//100
epochs = 100
n_labels = 2

x_train = tf.convert_to_tensor(x_train, dtype=tf.float32)
x_test = tf.convert_to_tensor(x_test, dtype=tf.float32)

y_train = tf.one_hot(y_train, n_labels)
y_test = tf.one_hot(y_test, n_labels)

## Model 1
model1 = tf.keras.Sequential(
  [
      tf.keras.layers.Input(shape=(n_features), batch_size = batch_size),
      tf.keras.layers.Dense(n_labels, activation='softmax')
  ])

model1.compile(optimizer='sgd',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

## Model 2
model2 = tf.keras.Sequential(
  [
      tf.keras.layers.Input(shape=(n_features), batch_size = batch_size),
      tf.keras.layers.Dense(10, activation='relu'),
      tf.keras.layers.Dense(5, activation='relu'),
      tf.keras.layers.Dense(n_labels, activation='softmax')
  ])

model2.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

## Model 3
model3 = tf.keras.Sequential(
  [
      tf.keras.layers.Input(shape=(n_features), batch_size = batch_size),
      tf.keras.layers.Dense(20, activation='relu'),
      tf.keras.layers.Dense(10, activation='relu'),
      tf.keras.layers.Dense(5, activation='relu'),
      tf.keras.layers.Dense(n_labels, activation='softmax')
  ])

model3.compile(optimizer='sgd',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# print model layers
model1.summary()
model2.summary()
model3.summary()


Model: "sequential_36"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_95 (Dense)            (300, 2)                  48        
                                                                 
Total params: 48 (192.00 Byte)
Trainable params: 48 (192.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Model: "sequential_37"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_96 (Dense)            (300, 10)                 240       
                                                                 
 dense_97 (Dense)            (300, 5)                  55        
                                                                 
 dense_98 (Dense)            (300, 2)                  12        
                                                            

In [74]:
steps_per_epoch = n_data//batch_size 
print("Steps per epoch: ", steps_per_epoch)
print("\n\n MODEL 1\n\n")

history1 = model1.fit(x = x_train, y = y_train , steps_per_epoch=steps_per_epoch, epochs=epochs, validation_steps=1)

print("\n\n MODEL 2\n\n")
history2 = model2.fit(x = x_train, y = y_train , steps_per_epoch=steps_per_epoch, epochs=epochs, validation_steps=1)

print("\n\n MODEL 3\n\n")
history3 = model3.fit(x = x_train, y = y_train , steps_per_epoch=steps_per_epoch, epochs=epochs, validation_steps=1)


Steps per epoch:  100


 MODEL 1


Epoch 1/100
100/100 [==============================] - 0s 1ms/step - loss: 0.7445 - accuracy: 0.5794
Epoch 2/100
100/100 [==============================] - 0s 2ms/step - loss: 0.5976 - accuracy: 0.7172
Epoch 3/100
100/100 [==============================] - 0s 1ms/step - loss: 0.5360 - accuracy: 0.7792
Epoch 4/100
100/100 [==============================] - 0s 1ms/step - loss: 0.5064 - accuracy: 0.7981
Epoch 5/100
100/100 [==============================] - 0s 1ms/step - loss: 0.4907 - accuracy: 0.8040
Epoch 6/100
100/100 [==============================] - 0s 1ms/step - loss: 0.4818 - accuracy: 0.8067
Epoch 7/100
100/100 [==============================] - 0s 1ms/step - loss: 0.4765 - accuracy: 0.8081
Epoch 8/100
100/100 [==============================] - 0s 1ms/step - loss: 0.4733 - accuracy: 0.8086
Epoch 9/100
100/100 [==============================] - 0s 1ms/step - loss: 0.4712 - accuracy: 0.8090
Epoch 10/100
100/100 [==============================] - 

In [75]:
## Model1
probabilities1 = model1.predict(x_test, steps=1)
probabilities2 = model2.predict(x_test, steps=1)
probabilities3 = model3.predict(x_test, steps=1)

1/1 [==============================] - 0s 62ms/step
